# Lab 1: Custom Messages
---

## Purpose
This lab will provide practice in creating custom messages. You will send a custom message over a topic that will result in a printed message and the TurtleBot3 to output one of two note progressions (one increasing in pitch while the other decreases in pitch)

## Report
Complete a short 2-3 page report that utilizes the format and answers the questions within the report template. The report template and an example report can be found within the Team under `Resources/Lab Template`.

## Turn-in Requirements
**[25 points]** Demonstration of keyboard control of USAFABot (preferably in person, but can be recorded and posted to Teams under the Lab1 channel).

**[50 points]** Report via Gradescope.

**[25 points]** Code: push your code to your repository. Also, include a screen shot of the **controller.py** and **usafabot_serial.py** files at the end of your report.